In [169]:
import os

os.chdir("/Users/sunilverma/Text-Summarizer-Project")
print(os.getcwd())



/Users/sunilverma/Text-Summarizer-Project


In [170]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True) 
class DataIngestionConfig:
    root_dir: Path 
    source_URL: str 
    local_data_file: Path 
    unzip_dir: Path  

In [180]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml, create_directories

In [ ]:
import sys
import importlib

# Clean module cache
modules_to_remove = [m for m in sys.modules if 'text_summarizer' in m]
for m in modules_to_remove:
    del sys.modules[m]

# Now import fresh
from text_summarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from text_summarizer.utils.common import read_yaml, create_directories

print(f"DEBUG - CONFIG_FILE_PATH from constants: {CONFIG_FILE_PATH}")
print(f"DEBUG - PARAMS_FILE_PATH from constants: {PARAMS_FILE_PATH}")

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
    ):
        print(f"DEBUG - ConfigurationManager received config_filepath: {config_filepath}")
        print(f"DEBUG - ConfigurationManager received params_filepath: {params_filepath}")
        
        # Convert relative paths to absolute
        if not config_filepath.is_absolute():
            config_filepath = Path(os.getcwd()) / config_filepath
        if not params_filepath.is_absolute():
            params_filepath = Path(os.getcwd()) / params_filepath
        
        print(f"DEBUG - After path conversion, config_filepath: {config_filepath}")
        print(f"DEBUG - After path conversion, params_filepath: {params_filepath}")
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir]) 
        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )
        return data_ingestion_config

In [173]:
import os 
import urllib.request as request 
import zipfile 
from text_summarizer.logging import logger 
from text_summarizer.utils.common import get_size


In [174]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config 

    def download_data(self): 
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file,
            )
            logger.info(f"File : {filename} downloaded with info: {headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"File extracted at : {unzip_path}")

In [186]:
try: 
    config = ConfigurationManager() 
    data_ingestion_config = config.get_data_ingestion_config() 
    data_ingestion = DataIngestion(config = data_ingestion_config) 
    data_ingestion.download_data()  # Fixed method name
    data_ingestion.extract_zip_file() 
except Exception as e: 
    raise e 

DEBUG - ConfigurationManager received config_filepath: config/config.yaml
DEBUG - ConfigurationManager received params_filepath: params.yaml
DEBUG - After path conversion, config_filepath: /Users/sunilverma/Text-Summarizer-Project/config/config.yaml
DEBUG - After path conversion, params_filepath: /Users/sunilverma/Text-Summarizer-Project/params.yaml
2025-12-17 07:07:52,648: INFO: common: yaml file: /Users/sunilverma/Text-Summarizer-Project/config/config.yaml loaded successfully
2025-12-17 07:07:52,655: INFO: common: yaml file: /Users/sunilverma/Text-Summarizer-Project/params.yaml loaded successfully
2025-12-17 07:07:52,658: INFO: common: created directory at: artifacts
2025-12-17 07:07:52,659: INFO: common: created directory at: artifacts/data_ingestion
2025-12-17 07:07:52,663: INFO: common: File: artifacts/data_ingestion/data.zip has size: 7718.35 KB
2025-12-17 07:07:52,664: INFO: 1196126151: File already exists of size: 7718.35 KB
2025-12-17 07:07:52,795: INFO: 1196126151: File extra